#### Install Pandas

In [ ]:
%pip install pandas

#### Memuat file `config.yaml`

In [ ]:
import yaml

with open('./config.yaml', 'r') as file:
    config = yaml.safe_load(file)

#### Mengimpor pandas

In [ ]:
import pandas as pd

#### Memuat dan menampilkan dataset

In [ ]:
dataframe = pd.read_csv(config['dataset'], low_memory=False)
dataframe.head()

#### Menampikan dimensi dataset


In [ ]:
dataframe.shape

#### Menampilkan daftar nama kolom

In [ ]:
dataframe.columns

#### Ringkasan data

In [ ]:
dataframe.describe()

#### Menampilkan info dataset

In [ ]:
dataframe.info()

#### Membersihkan dataset yang null dan menyimpan pada `df_clean_rows`

In [ ]:
dataframe.drop(dataframe[dataframe['hosp_yn'] == 'Missing'].index, inplace=True)
dataframe.drop(dataframe[dataframe['icu_yn'] == 'Missing'].index, inplace=True)
dataframe.drop(dataframe[dataframe['death_yn'] == 'Missing'].index, inplace=True)
dataframe.drop(dataframe[dataframe['medcond_yn'] == 'Missing'].index, inplace=True)
dataframe.drop(dataframe[dataframe['Race and ethnicity (combined)'] == 'Missing'].index, inplace=True)
dataframe.drop(dataframe[dataframe['age_group'] == 'Unknown'].index, inplace=True)
dataframe.drop(dataframe[dataframe['Race and ethnicity (combined)'] == 'Unknown'].index, inplace=True)
df_clean_rows = dataframe.dropna()

#### Menampilkan dataset yang bersih

In [ ]:
df_clean_rows

#### Install matplotlib

In [ ]:
%pip install matplotlib

#### Mengimpor matplotlib

In [ ]:
import matplotlib.pyplot as plt

#### Install seaborn

In [ ]:
%pip install seaborn

#### Mengimpor seaborn

In [ ]:
import seaborn as sns

#### Menampilkan data kolom `age_group`

In [ ]:
plt.figure(figsize = (30,4))
sns.countplot(x='age_group', data=df_clean_rows)
plt.show()

#### Menampilkan data kolom `Race and ethnicity (combined)`

In [ ]:
plt.figure(figsize = (30,4))
sns.countplot(x='Race and ethnicity (combined)', data=df_clean_rows)
plt.show()

#### Install sklearn

In [ ]:
%pip install sklearn

#### Mengimpor sklearn

In [ ]:
from sklearn.preprocessing import LabelEncoder

#### Meng-encode label string ke integer

In [ ]:
columns_to_encode = [
    'cdc_report_dt', 'pos_spec_dt', 'onset_dt', 'current_status', 'sex',
    'age_group', 'Race and ethnicity (combined)', 'hosp_yn', 'icu_yn',
    'medcond_yn', 'death_yn'
]

le = LabelEncoder()

for column in columns_to_encode:
    df_clean_rows.loc[:, column] = le.fit_transform(df_clean_rows[column])

#### Menampilkan dataset yang sudah di-encode

In [ ]:
df_clean_rows

## Machine learning model

#### Memilih target

In [ ]:
y = df_clean_rows['cdc_report_dt']
y

#### Memilih fitur

In [ ]:
features = ['pos_spec_dt', 'onset_dt', 'sex', 'age_group', 'Race and ethnicity (combined)', 'hosp_yn', 'icu_yn', 'medcond_yn', 'death_yn']
X = df_clean_rows[features]
X

#### Menampilkan fitur

In [ ]:
X.describe()

#### Mengimpor `DecisionTreeRegresor`

In [ ]:
from sklearn.tree import DecisionTreeRegressor

#### Konfigurasi model

In [ ]:
covid_model = DecisionTreeRegressor(random_state=1)

#### Training model

In [ ]:
covid_model.fit(X, y)

#### Melakukan prediksi

In [ ]:
covid_model.predict(X.head())

In [ ]:
y.head()

## Evaluasi Model

#### Mengimpor evaluation metric

In [ ]:
from sklearn.metrics import mean_absolute_error

In [ ]:
y_hat = covid_model.predict(X)
mean_absolute_error(y, y_hat)

#### Training dan testing

In [ ]:
from sklearn.model_selection import train_test_split

#### Membagi dataset menjadi dua bagian

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

#### Konfigurasi dan training model

In [ ]:
covid_model = DecisionTreeRegressor(random_state=1)
covid_model.fit(X_train, y_train)

#### Evaluasi model

In [ ]:
y_hat = covid_model.predict(X_test)
mean_absolute_error(y_test, y_hat)

#### Optimasi model

In [ ]:
def get_mae(max_leaf_nodes, X_train, X_test, y_train, y_test):
    model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=0)
    model.fit(X_train, y_train)
    y_hat = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_hat)
    return mae

In [ ]:
for max_leaf_nodes in [5, 50, 500, 5000]:
    leaf_mae = get_mae(max_leaf_nodes, X_train, X_test, y_train, y_test)
    print(f'Max leaf nodes: {max_leaf_nodes} \t Mean Absolute Error: {int(leaf_mae)}')

#### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
rf_model = RandomForestRegressor(n_estimators=100, random_state=1)
rf_model.fit(X_train, y_train)
y_hat = rf_model.predict(X_test)
print(f'Mean Absolute Error: {int(mean_absolute_error(y_test, y_hat))}')